In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [68]:
from google.colab import files
f=files.upload()

Saving creative_writing2.csv to creative_writing2 (1).csv


In [69]:
df = pd.read_csv('creative_writing2 (1).csv')

In [70]:
df.dropna(inplace=True)

In [71]:
corpus = [(prompt, creative_writing) for prompt, creative_writing in zip(df['prompt'], df['creative_writing'])]

In [72]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([row[1] for row in corpus])
total_words = len(tokenizer.word_index) + 1

In [73]:
input_sequences = []
for _, creative_writing in corpus:
    token_list = tokenizer.texts_to_sequences([creative_writing])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [74]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [75]:
X, y = input_sequences[:,:-1],input_sequences[:,-1]

In [76]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))

In [77]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [78]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
50/50 [==============================] - 12s 145ms/step - loss: 5.7510 - accuracy: 0.1192
Epoch 2/100
50/50 [==============================] - 6s 120ms/step - loss: 5.2335 - accuracy: 0.1408
Epoch 3/100
50/50 [==============================] - 7s 141ms/step - loss: 5.1332 - accuracy: 0.1408
Epoch 4/100
50/50 [==============================] - 6s 120ms/step - loss: 5.0028 - accuracy: 0.1439
Epoch 5/100
50/50 [==============================] - 7s 148ms/step - loss: 4.8496 - accuracy: 0.1687
Epoch 6/100
50/50 [==============================] - 7s 134ms/step - loss: 4.6747 - accuracy: 0.1959
Epoch 7/100
50/50 [==============================] - 7s 128ms/step - loss: 4.4763 - accuracy: 0.2169
Epoch 8/100
50/50 [==============================] - 7s 130ms/step - loss: 4.2489 - accuracy: 0.2391
Epoch 9/100
50/50 [==============================] - 7s 129ms/step - loss: 4.0238 - accuracy: 0.2467
Epoch 10/100
50/50 [==============================] - 6s 130ms/step - loss: 3.8093 - accur

In [79]:
def generate_text(prompt, model, tokenizer, max_sequence_len, num_words=50):
    seed_text = prompt.lower()
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text.capitalize())

In [84]:
while True:
    prompt = input("Enter a prompt for writing (type 'exit' to quit):\n")
    if prompt.lower() == 'exit':
        print("Goodbye!")
        break
    print("Here's your writing:")
    generate_text(prompt, model, tokenizer, max_sequence_len)
    print("\n")

Enter a prompt for writing (type 'exit' to quit):
A beautiful sunset
Here's your writing:
A beautiful sunset sunset paint the sky in shades of orange and gold with clouds ablaze and the horizon ablaze with color i felt a sense of gratitude for the beauty of the world and the gift of another day day day patterns of another day day day day day day day day


Enter a prompt for writing (type 'exit' to quit):
About an adventurous journey
Here's your writing:
About an adventurous journey through the shimmering portal i found myself transported to a world of wonder and enchantment twisting where me and fantasy intertwined reality i had entered a world untouched by man i had been transported to a paradise on earth wonder and wonder to the embrace of nature and and possible


Enter a prompt for writing (type 'exit' to quit):
Exploration
Here's your writing:
Exploration the tranquility of the forest i embarked on a peaceful journey guided by the whispers of the wind and lurked in the rhythm of t